In [1]:
import os

In [2]:
pwd

'd:\\PW_DS\\NLP\\End-To-End-Text-Summarizer-Project-using-Transformer\\research'

In [3]:
os.chdir('../')

In [4]:
pwd

'd:\\PW_DS\\NLP\\End-To-End-Text-Summarizer-Project-using-Transformer'

## Entity
### Set params.yaml for paramerts 

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int



## Config Manager

In [6]:
from Text_Summarizer.constants import *
from Text_Summarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfiguraitonManager:
    def __init__(
        self,
        config_file_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH):

        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    
    def get_model_trainer_config(self):
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt=config.model_ckpt,
            num_train_epochs=params.num_train_epochs,
            warmup_steps=params.warmup_steps,
            per_device_train_batch_size=params.per_device_train_batch_size,
            weight_decay=params.weight_decay,
            logging_steps=params.logging_steps,
            evaluation_strategy=params.evaluation_strategy,
            eval_steps=params.eval_steps,
            save_steps=params.save_steps,
            gradient_accumulation_steps=params.gradient_accumulation_steps
        )

        return model_trainer_config

## Components

In [8]:
from datasets import load_dataset, load_from_disk

from transformers import AutoModelForSeq2SeqLM,   AutoTokenizer
from transformers import DataCollatorForSeq2Seq
from transformers import TrainingArguments, Trainer

import torch

!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

c:\Users\Maaz\anaconda3\envs\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-01-11 20:21:46,395: INFO: config: PyTorch version 2.1.2 available.]
Found existing installation: transformers 4.36.2
Uninstalling transformers-4.36.2:
  Successfully uninstalled transformers-4.36.2
Found existing installation: accelerate 0.26.0
Uninstalling accelerate-0.26.0:
  Successfully uninstalled accelerate-0.26.0
  Using cached transformers-4.36.2-py3-none-any.whl.metadata (126 kB)
  Using cached accelerate-0.26.0-py3-none-any.whl.metadata (18 kB)
Using cached transformers-4.36.2-py3-none-any.whl (8.2 MB)
Using cached accelerate-0.26.0-py3-none-any.whl (270 kB)


In [9]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        device = 'cuda' if torch.cuda.is_available() else 'cpu' ## To check GPU is connected or not
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer,model=model_pegasus)

        # Load Dataset
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs=self.config.num_train_epochs,
            per_device_train_batch_size=self.config.per_device_train_batch_size,
            weight_decay=self.config.weight_decay,
            logging_steps=self.config.logging_steps,
            evaluation_strategy=self.config.evaluation_strategy,
            eval_steps=self.config.eval_steps,
            save_steps=1e6,
            gradient_accumulation_steps=self.config.gradient_accumulation_steps
        )

        # trainer_args = TrainingArguments(
        #     output_dir = 'pegasus-samsum',
        #     num_train_epochs = 1,
        #     warmup_steps = 500,
        #     per_device_train_batch_size = 1,
        #     per_device_eval_batch_size = 1,
        #     weight_decay = 0.01,
        #     logging_steps = 10,
        #     evaluation_strategy = 'steps',
        #     eval_steps = 500,
        #     save_steps = 1e6,
        #     gradient_accumulation_steps = 16
        # )


        trainer = Trainer(
            model=model_pegasus,
            args=trainer_args,
            data_collator=seq2seq_data_collator,
            tokenizer=tokenizer,
            train_dataset = dataset_samsum_pt['test'], ## Change to train 
            eval_dataset = dataset_samsum_pt['validation'] 
        )
        
         
        trainer.train()



## Pipeline

In [10]:
try:
    config = ConfiguraitonManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()

except Exception as e:
    raise e

[2024-01-11 20:22:25,466: INFO: common: yaml file: config\config.yaml loaed successfully.]
[2024-01-11 20:22:25,476: INFO: common: yaml file: params.yaml loaed successfully.]
[2024-01-11 20:22:25,480: INFO: common: directory: artifacts created successfully.]
[2024-01-11 20:22:25,482: INFO: common: directory: artifacts/model_trainer created successfully.]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  2%|▏         | 1/51 [04:19<3:36:07, 259.36s/it]

KeyboardInterrupt: 